In [1]:
#import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.utils import resample
from nltk.stem.wordnet import WordNetLemmatizer

#sklearn package 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn. preprocessing import LabelEncoder
from sklearn.naive_bayes import ComplementNB,MultinomialNB,GaussianNB 

from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score,classification_report, confusion_matrix,recall_score,precision_score,f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Read the data

df = pd.read_json('News_Category_Dataset_v3.json', lines=True)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [3]:
df.shape

(209527, 6)

In [4]:
df.describe()

C:\Users\win10\AppData\Local\Temp\ipykernel_1420\3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,link,headline,category,short_description,authors,date
count,209527,209527,209527,209527,209527,209527
unique,209486,207996,42,187022,29169,3890
top,https://www.huffingtonpost.comhttps://www.wash...,Sunday Roundup,POLITICS,,,2014-03-25 00:00:00
freq,2,90,35602,19712,37418,100
first,NaN,NaN,NaN,NaN,NaN,2012-01-28 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2022-09-23 00:00:00


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [6]:
# Drop unwanted columns 

df.drop(['link','date'],axis=1,inplace = True)
df.head()

,headline,category,short_description,authors
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP"
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski


In [7]:
# Total number of unique categories

df['category'].nunique()

42

In [8]:
# count per category

df['category'].value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [9]:
# Count of duplicate values

df.duplicated().sum()

447

In [10]:
# Delete the dublicate values

df.drop_duplicates(keep='last',inplace=True)

In [11]:
# Check for duplicate headline and short description

df.duplicated(subset=['headline', 'short_description']).sum()

42

In [12]:
df.drop_duplicates(subset=['headline', 'short_description'],inplace=True,keep='last')

In [13]:
df.shape

(209038, 4)

In [14]:
# Null values check

df.isnull().sum()

headline             0
category             0
short_description    0
authors              0
dtype: int64

In [15]:
# Blank values check

df[df['headline'] == '']

,headline,category,short_description,authors
95567,,RELIGION,Let everyone not wrapped in tired and disprove...,"Matthew Fox, ContributorRadical theologian Mat..."
122145,,QUEER VOICES,,"Gary Snyder, ContributorWriter and Media Strat..."


In [16]:
# Drop the blank values

headline_blank = df['headline'] == ''
df = df[~headline_blank]


In [17]:
# Check and drop the blank short describtion column

df[df['short_description'] == '']

,headline,category,short_description,authors
20773,The Big Blue Wave,COMEDY,,"Shan Wells, ContributorSciency sun venerator +..."
21523,"Inside Rukban Camp, One Of Syria’s Most Desper...",WORLD NEWS,,"Yasser Allawi, Syria Deeply"
22793,Syrian Refugees Return From Lebanon Only To Fl...,WORLD NEWS,,"Abby Sewell, Refugees Deeply"
32223,Your Guide To The Best BBQ In St. Louis Accord...,TASTE,,
34488,The Bechdel Test,COMEDY,,"Hilary Fitzgerald Campbell, ContributorHilary'..."
...,...,...,...,...
206924,True LGBTQ Stories: When A College Football Te...,QUEER VOICES,,"Nathan Manske, Contributor\nExecutive Director..."
206939,'Chasing Ice' -- The Climate Change Debate Is ...,ENVIRONMENT,,"Greg Reitman, Contributor\nFounder of Blue Wat..."
208803,Practice for International Kissing Day: 'Love ...,IMPACT,,"Karen Dalton-Beninato, Contributor\nWriter Nea..."
208865,"Zach Huston, Victim of Brutal Ohio School Bull...",QUEER VOICES,,"Ian Thompson, Contributor\nLegislative Represe..."


In [18]:
description_blank = df['short_description'] == ''
df = df[~description_blank]

In [19]:
df['category'].value_counts()

POLITICS          32425
WELLNESS          17938
ENTERTAINMENT     14772
STYLE & BEAUTY     9665
TRAVEL             9418
PARENTING          8762
FOOD & DRINK       6331
QUEER VOICES       5601
HEALTHY LIVING     5264
BUSINESS           5130
COMEDY             4642
SPORTS             4414
HOME & LIVING      4282
BLACK VOICES       4174
THE WORLDPOST      3664
WEDDINGS           3653
PARENTS            3516
DIVORCE            3423
WORLD NEWS         3295
WOMEN              3136
IMPACT             3085
CRIME              2831
MEDIA              2402
WEIRD NEWS         2316
TECH               2054
GREEN              2044
TASTE              1940
RELIGION           1878
SCIENCE            1803
MONEY              1755
STYLE              1564
ENVIRONMENT        1441
U.S. NEWS          1377
ARTS & CULTURE     1338
WORLDPOST          1242
CULTURE & ARTS     1062
FIFTY              1042
GOOD NEWS          1039
LATINO VOICES      1022
COLLEGE             921
EDUCATION           902
ARTS            

In [20]:
# Drop the category type having less number of records to improve accuracy. dropping categories having count < 2000

categories_keep = (df['category'] == 'POLITICS') | (df['category'] == 'WELLNESS' ) | (df['category'] == 'ENTERTAINMENT') | (df['category'] == 'TRAVEL') | \
             (df['category'] == 'STYLE & BEAUTY') | (df['category'] == 'PARENTING' ) | (df['category'] == 'HEALTHY LIVING') | (df['category'] == 'QUEER VOICES') | \
             (df['category'] == 'FOOD & DRINK') | (df['category'] == 'BUSINESS' ) | (df['category'] == 'COMEDY') | (df['category'] == 'SPORTS') | (df['category'] == 'BLACK VOICES')  | \
             (df['category'] == 'HOME & LIVING' ) | (df['category'] == 'PARENTS') | (df['category'] == 'THE WORLDPOST') | (df['category'] == 'WEDDINGS') | (df['category'] == 'WOMEN') 
#              (df['category'] == 'CRIME') | (df['category'] == 'IMPACT') | (df['category'] == 'DIVORCE') | (df['category'] == 'WORLD NEWS') | (df['category'] == 'MEDIA') | (df['category'] == 'WEIRD NEWS') | (df['category'] == 'GREEN') | \
#              (df['category'] == 'TECH') 
df = df[categories_keep]

In [21]:
category = df['category'].value_counts()
category

POLITICS          32425
WELLNESS          17938
ENTERTAINMENT     14772
STYLE & BEAUTY     9665
TRAVEL             9418
PARENTING          8762
FOOD & DRINK       6331
QUEER VOICES       5601
HEALTHY LIVING     5264
BUSINESS           5130
COMEDY             4642
SPORTS             4414
HOME & LIVING      4282
BLACK VOICES       4174
THE WORLDPOST      3664
WEDDINGS           3653
PARENTS            3516
WOMEN              3136
Name: category, dtype: int64

In [22]:
data_1 = df[df['category'] == 'POLITICS']
data_1 = resample(data_1, replace=False, n_samples=3000, random_state=123)
data_2 = df[df['category'] == 'WELLNESS']
data_2 = resample(data_2, replace=False, n_samples=3000, random_state=123)
data_3 = df[df['category'] == 'ENTERTAINMENT']
data_3 = resample(data_3, replace=False, n_samples=3000, random_state=123)
data_4 = df[df['category'] == 'TRAVEL']
data_4 = resample(data_4, replace=False, n_samples=3000, random_state=123)
data_5 = df[df['category'] == 'STYLE & BEAUTY']
data_5 = resample(data_5, replace=False, n_samples=3000, random_state=123)
data_6 = df[df['category'] == 'PARENTING']
data_6 = resample(data_6, replace=False, n_samples=3000, random_state=123)
data_7 = df[df['category'] == 'HEALTHY LIVING']
data_7 = resample(data_7, replace=False, n_samples=3000, random_state=123)
data_8 = df[df['category'] == 'QUEER VOICES']
data_8 = resample(data_8, replace=False, n_samples=3000, random_state=123)
data_9 = df[df['category'] == 'FOOD & DRINK']
data_9 = resample(data_9, replace=False, n_samples=3000, random_state=123)
data_10 = df[df['category'] == 'BUSINESS']
data_10 = resample(data_10, replace=False, n_samples=3000, random_state=123)
data_11 = df[df['category'] == 'COMEDY']
data_11 = resample(data_11, replace=False, n_samples=3000, random_state=123)
data_12= df[df['category'] == 'PARENTS']
data_12 = resample(data_12, replace=False, n_samples=3000, random_state=123)
data_13= df[df['category'] == 'SPORTS']
data_13 = resample(data_13, replace=False, n_samples=3000, random_state=123)
data_14 = df[df['category'] == 'HOME & LIVING']
data_14 = resample(data_14, replace=False, n_samples=3000, random_state=123)
data_15 = df[df['category'] == 'BLACK VOICES']
data_15 = resample(data_15, replace=False, n_samples=3000, random_state=123)

category_list = [data_1 , data_2 , data_3 , data_4 ,data_5 , data_6 , data_7, data_8 , data_9 , data_10, data_11 , data_12 , data_13, data_14 ,data_15]

df = pd.concat(category_list)
df['category'].value_counts()

POLITICS          3000
WELLNESS          3000
ENTERTAINMENT     3000
TRAVEL            3000
STYLE & BEAUTY    3000
PARENTING         3000
HEALTHY LIVING    3000
QUEER VOICES      3000
FOOD & DRINK      3000
BUSINESS          3000
COMEDY            3000
PARENTS           3000
SPORTS            3000
HOME & LIVING     3000
BLACK VOICES      3000
Name: category, dtype: int64

In [23]:
# Combining columns headline and short_description

df['text'] = df['headline'] + '-' + df['short_description']

In [24]:
df.head()

,headline,category,short_description,authors,text
64888,Iowa Senator Doesn’t Want Anyone Forcing Soldi...,POLITICS,Joni Ernst considers lunch a matter of nationa...,Jacquie Lee,Iowa Senator Doesn’t Want Anyone Forcing Soldi...
89468,Heroin Epidemic Stars In Conservative Ad Again...,POLITICS,"Union president says he's ""literally disgusted...",Jason Cherkis,Heroin Epidemic Stars In Conservative Ad Again...
61502,Trump Eyes Fracking Mogul Harold Hamm As Energ...,POLITICS,Hamm would add to Trump's roster of fossil fue...,,Trump Eyes Fracking Mogul Harold Hamm As Energ...
42004,Hands Off Medicare And Medicaid Tour: Find Thr...,POLITICS,Video by Portland Press Herald Nobody voted to...,"Alex Lawson and Jon ""Bowzer"" Bauman, Contributors",Hands Off Medicare And Medicaid Tour: Find Thr...
1978,Boxer-Senator Manny Pacquiao To Run For Philip...,POLITICS,"“I am a fighter, and I will always be a fighte...",,Boxer-Senator Manny Pacquiao To Run For Philip...


In [25]:
# Drop the unwanted columns

df.drop(['authors','headline','short_description'],axis=1,inplace=True)

In [26]:
df.head()

,category,text
64888,POLITICS,Iowa Senator Doesn’t Want Anyone Forcing Soldi...
89468,POLITICS,Heroin Epidemic Stars In Conservative Ad Again...
61502,POLITICS,Trump Eyes Fracking Mogul Harold Hamm As Energ...
42004,POLITICS,Hands Off Medicare And Medicaid Tour: Find Thr...
1978,POLITICS,Boxer-Senator Manny Pacquiao To Run For Philip...


In [27]:
df.shape

(45000, 2)

In [28]:
# clean the text data using regex and data cleaning function
def data_cleaning(text):
    whitespace = re.compile(r"\s+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = whitespace.sub(' ', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]","", text)
    text = re.sub("\d+", "", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    text = text.lower()
    
    # removing stop-words
    text = [word for word in text.split() if word not in list(stopwords.words('english'))]
    
    # word lemmatization
    sentence = []
    for word in text:
        lemmatizer = WordNetLemmatizer()
        sentence.append(lemmatizer.lemmatize(word,'v'))
        
    return ' '.join(sentence) 

In [29]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
# checking result of data_cleaning()

print("Text before pre-processing:\n",df['text'][1978])
print("\n Text after pre-processing:\n",data_cleaning(df['text'][1978]))

Text before pre-processing:
 Boxer-Senator Manny Pacquiao To Run For Philippine President-“I am a fighter, and I will always be a fighter inside and outside the ring,” said the 42-year-old boxing icon.

 Text after pre-processing:
 boxersenator manny pacquiao run philippine presidenti fighter always fighter inside outside ring say yearold box icon


In [31]:
# Apply data_cleaning function to column 'text'

df['text'] = df['text'].apply(data_cleaning)

In [32]:
df.head()

,category,text
64888,POLITICS,iowa senator doesnt want anyone force soldier ...
89468,POLITICS,heroin epidemic star conservative ad new hamps...
61502,POLITICS,trump eye fracking mogul harold hamm energy se...
42004,POLITICS,hand medicare medicaid tour find three republi...
1978,POLITICS,boxersenator manny pacquiao run philippine pre...


In [33]:
#train and test split

X = df['text']

#label encoding the target

label = LabelEncoder()
y = label.fit_transform(df['category'])

#split the train and test dataset

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state=2)

In [34]:
y

array([9, 9, 9, ..., 0, 0, 0])

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40500,)
(4500,)
(40500,)
(4500,)


In [36]:
#tfidf vectorizer

vecto =  TfidfVectorizer(stop_words='english',max_df = 0.99,min_df=10,
                                   ngram_range=(1, 2),lowercase=True, max_features=5000)
vecto = vecto.fit(X_train)

X_train = vecto.transform(X_train).toarray()
X_test = vecto.transform(X_test).toarray()
X_train.shape

(40500, 5000)

In [37]:
print(vecto.get_feature_names())

['aaron', 'abandon', 'abc', 'abcs', 'ability', 'able', 'abortion', 'abroad', 'absolute', 'absolutely', 'absurd', 'abundance', 'abuse', 'abusive', 'academy', 'academy award', 'accent', 'accept', 'acceptable', 'acceptance', 'access', 'accessories', 'accessory', 'accident', 'accidentally', 'accompany', 'accomplish', 'accord', 'account', 'accurate', 'accuse', 'achieve', 'achievement', 'acknowledge', 'acquire', 'act', 'act like', 'action', 'active', 'activism', 'activist', 'activists', 'activities', 'activity', 'actor', 'actors', 'actress', 'actual', 'actually', 'acupuncture', 'ad', 'adam', 'adams', 'adapt', 'add', 'addict', 'addiction', 'addition', 'additional', 'address', 'adele', 'adjust', 'administration', 'administrations', 'admire', 'admit', 'adopt', 'adoption', 'adorable', 'ads', 'adult', 'adulthood', 'adults', 'advance', 'advantage', 'adventure', 'advertise', 'advice', 'advise', 'advocacy', 'advocate', 'affair', 'affect', 'afford', 'affordable', 'affordable care', 'afghanistan', 'af

C:\Users\win10\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [38]:
tfidf_df = pd.DataFrame(X_train,columns = vecto.get_feature_names())
tfidf_df.head(4)

,aaron,abandon,abc,abcs,ability,able,abortion,abroad,absolute,absolutely,...,youtube,youve,zealand,zen,zero,zika,zika virus,zimmerman,zoe,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# Training the model

# Naibe Bayes - Multinominal

multinb = MultinomialNB()
multinb.fit(X_train , y_train)

y_pred = multinb.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test,y_pred))
print("Recall score : ",recall_score(y_test,y_pred, average='macro'))
print("Precision : ",precision_score(y_test,y_pred, average='macro'))
print("F1 score : ",f1_score(y_test,y_pred, average='macro'))

Accuracy Score:  0.6291111111111111
Recall score :  0.6250322146123981
Precision :  0.6289819465076026
F1 score :  0.6255312315033367


In [41]:
# Naibe Bayes - Complement 

compnb = ComplementNB(alpha=1.0)
compnb.fit(X_train , y_train)

y_pred = compnb.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test,y_pred))
print("Recall score : ",recall_score(y_test,y_pred, average='macro'))
print("Precision : ",precision_score(y_test,y_pred, average='macro'))
print("F1 score : ",f1_score(y_test,y_pred, average='macro'))

Accuracy Score:  0.628
Recall score :  0.6219901002052103
Precision :  0.6173508415320553
F1 score :  0.6144785357992929


In [47]:
# Logistic Regression

import warnings
from sklearn.exceptions import ConvergenceWarning

log_reg = LogisticRegression(max_iter=1000)

# log_reg.fit(X_train , y_train)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    log_reg.fit(X_train, y_train)
    
y_pred = log_reg.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test,y_pred))
print("Recall score : ",recall_score(y_test,y_pred, average='macro'))
print("Precision : ",precision_score(y_test,y_pred, average='macro'))
print("F1 score : ",f1_score(y_test,y_pred, average='macro'))

Accuracy Score:  0.6415555555555555
Recall score :  0.6373725682983397
Precision :  0.6382884469582847
F1 score :  0.6371700926005767


In [46]:
# Training the logistic regression model for 10 epochs

log_reg = LogisticRegression(solver='lbfgs', max_iter=100)
for epoch in range(10):
#     log_reg.fit(X_train, y_train)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
        log_reg.fit(X_train, y_train)
    
    print("Epoch {}: Training Accuracy = {:.4f}, Testing Accuracy = {:.4f}".format(
        epoch+1, log_reg.score(X_train, y_train), log_reg.score(X_test, y_test)))

Epoch 1: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 2: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 3: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 4: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 5: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 6: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 7: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 8: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 9: Training Accuracy = 0.7511, Testing Accuracy = 0.6420
Epoch 10: Training Accuracy = 0.7511, Testing Accuracy = 0.6420


In [48]:
y_pred = log_reg.predict(X_test)

print("Accuracy Score: ", accuracy_score(y_test,y_pred)) 
print("Recall score : ",recall_score(y_test,y_pred, average='macro'))
print("Precision : ",precision_score(y_test,y_pred, average='macro'))
print("F1 score : ",f1_score(y_test,y_pred, average='macro'))

Accuracy Score:  0.6415555555555555
Recall score :  0.6373725682983397
Precision :  0.6382884469582847
F1 score :  0.6371700926005767
